# Spikeinterface Cookbook


In [ ]:
!pip install spikeinterface

## Playing with Recording and Sorting objects

#### Generate synthetic data

In [ ]:
from spikeinterface.core import generate_recording 

recording = generate_recording(num_channels=3, durations=[10])
recording

In [ ]:
from spikeinterface.core import generate_sorting 

sorting = generate_sorting(num_units=3, durations=[10])
sorting

#### Setting channel names and unit IDs

In [ ]:
recording = recording.rename_channels(new_channel_ids=["a", "b", "c"])  # This is not in-place
recording.get_channel_ids()

In [ ]:
sorting = sorting.rename_units(new_unit_ids=["unit1", "unit2", "unit3"])  # This is not in-place
sorting.get_unit_ids()

### Slicing a Recording and Sorting Objects

#### Channels

In [ ]:
channel_sliced_recording = recording.channel_slice(channel_ids=["a", "b"])
channel_sliced_recording

#### Units

In [ ]:
unit_selected_sorting = sorting.select_units(unit_ids=["unit1", "unit2"])
unit_selected_sorting

### Frames / Time

In [ ]:
sliced_recording = recording.frame_slice(start_frame=0, end_frame=1000)
sliced_recording

In [ ]:
sliced_sorting = sorting.frame_slice(start_frame=0, end_frame=1000)
sliced_sorting

### Combining recordings

#### Concatenating recordings (across time)

In [ ]:
from spikeinterface.core import concatenate_recordings

recording1 = generate_recording(num_channels=3, durations=[10])
recording2 = generate_recording(num_channels=3, durations=[10])

concanted_recordings = concatenate_recordings([recording1, recording2])

assert concanted_recordings.get_duration() == recording1.get_duration()  + recording2.get_duration()


#### Aggregating channels to a single recording

In [ ]:
from spikeinterface.core import aggregate_channels

recording1 = generate_recording(num_channels=3, durations=[10], set_probe=False)  # To avoid location check
recording1 = recording1.rename_channels(new_channel_ids=["a", "b", "c"])
recording2 = generate_recording(num_channels=2, durations=[10], set_probe=False)  
recording2 = recording2.rename_channels(new_channel_ids=["d", "e"])

aggregated_recording = aggregate_channels([recording1, recording2])  
assert aggregated_recording.get_num_channels() == 5
assert list(aggregated_recording.get_channel_ids()) == ['a', 'b', 'c', 'd', 'e']  # Failing right now

### Summary of operations




![Recording Operations](./recording_operations.png)


## Parallel Processing

### What do we parallelize over

![Chuking Description](./parallel_processing.png)

### Parameters to control paralell execution 

* lenght of the chunk
    * chunk_duration :  Lenght of the chunk in seconds
    * chunk_size: Number of samples per chunk
    * chunk_memory: Memory usage for each job (e.g. "100M", "1G")
    * total_memory Total memory usage (e.g. "500M", "2G")
* n_jobs: Number of jobs to use. With -1 the number of jobs is the same as number of cores
* progress_bar: Whether to show a progress bar
* mp_context: fork, span or forkserver

![job_kwargs](./job_kwargs.png)

### Setting global job_kwargs

In [ ]:
sorting = generate_sorting(num_units=3, durations=[10])

## Saving Recording and Sorting Objects

### Binary format

#### Recording

In [ ]:
from pathlib import Path
from spikeinterface.core import generate_recording

recording = generate_recording(num_channels=3, durations=[10], set_probe=True)
recording = recording.rename_channels(new_channel_ids=["a", "b", "c"])  # This is not in-place
recording.set_property("a_property", ["value1", "value2", "value3"])  # This is in place


folder_path = Path("./test_recording")

job_kwargs={'progress_bar': True, "verbose":True, "n_jobs":2}
binary_recording = recording.save_to_folder(folder=folder_path,  overwrite=True, **job_kwargs)   
binary_recording

In [ ]:
recording

#### Sorting

In [ ]:
from pathlib import Path
from spikeinterface.core import generate_sorting

sorting = generate_sorting(num_units=3, durations=[10])
sorting = sorting.rename_units(new_unit_ids=["unit1", "unit2", "unit3"])  # This is not in-place

sorting.set_property("a_property", ["value1", "value2", "value3"])  # This is in place


folder_path = Path("./test_sorting")

job_kwargs={'progress_bar': True, "verbose":True, "n_jobs":2}
binary_sorting = sorting.save_to_folder(folder=folder_path,  overwrite=True, **job_kwargs)   
binary_sorting


In [ ]:
sorting

### Zarr format

#### Recording

In [ ]:
from pathlib import Path
from spikeinterface.core import generate_recording

recording = generate_recording(num_channels=3, durations=[10], set_probe=True)
recording = recording.rename_channels(new_channel_ids=["a", "b", "c"])  # This is not in-place
recording.set_property("a_property", ["value1", "value2", "value3"])  # This is in place

folder_path = Path("./test_recording.zarr")

job_kwargs={'progress_bar': True, "verbose":True, "n_jobs":2}
zarr_recording = recording.save_to_zarr(folder=folder_path,  overwrite=True, **job_kwargs)   
zarr_recording

In [ ]:
zarr_recording._root.tree()

### Sorting

In [ ]:
from pathlib import Path
from spikeinterface.core import generate_sorting

sorting = generate_sorting(num_units=3, durations=[10])
sorting = sorting.rename_units(new_unit_ids=["unit1", "unit2", "unit3"])  # This is not in-place

sorting.set_property("a_property", ["value1", "value2", "value3"])  # This is in place


folder_path = Path("./test_sorting.zarr")

job_kwargs={'progress_bar': True, "verbose":True, "n_jobs":2}
zarr_sorting = sorting.save_to_zarr(folder=folder_path,  overwrite=True, **job_kwargs)   
zarr_sorting


In [ ]:
zarr_sorting._root.tree()